In [ ]:
import copy
import h5py
import itertools
import numpy as np
import os
import scipy
import scipy.interpolate
import tqdm

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import palettable

In [ ]:
import yt
import trident
import unyt as u

In [ ]:
import kalepy as kale
import verdict

In [ ]:
import one_zone

# Parameters

In [ ]:
# Analysis parameters
seed = 15482
np.random.seed( seed )
rng = np.random.default_rng( seed )
load_existing_sightlines = True
verbose = False

In [ ]:
# Data management parameters
distribution_fp = './data/EAGLE/histogram_galaxies_logM200c-Msun-12.0-12.5_200_seed0_hneutralssh.hdf5'
data_dir = './data/synthetic_data/sample1'
observer_data_dir = './data/synthetic_data_samples/sample1'

In [ ]:
# Ray parameters
redshift = 0.25
n_sightlines = 100
min_clouds_per_sightline = 1
max_clouds_per_sightline = 3
velocity_range = [ -150., 150. ] # In km/s
finite_cloud_max_logT = 5 # We'll only allow one cloud per line of sight with temperatures greater than this

In [ ]:
# Spectra parameters
ions = [
    'H I',
    'O I',
    'C II',
    'C III',
    'C IV',
    'N II',
    'N III',
    'Si II',
    'Si III',
    'Si IV',
    'N V',
    'O VI',
    'Mg II'
]
fields = [
    'H_p0_number_density', 
    'O_p0_number_density',
    'C_p1_number_density',
    'C_p2_number_density',
    'C_p3_number_density',
    'N_p1_number_density',
    'N_p2_number_density',
    'Si_p1_number_density',
    'Si_p2_number_density',
    'Si_p3_number_density',
    'N_p4_number_density',
    'O_p5_number_density',
    'Mg_p1_number_density'
]
snr = 30

# Load and Format Data

Units are [ g/cm^3, mass fraction, logK, log cm^-2 ] respectively for ['Density', 'Metallicity', 'Temperature', 'hneutralssh'].

In [ ]:
f = h5py.File( distribution_fp, 'r' )

In [ ]:
# Get centers, reformat edges

histogram_axes = [ 'Temperature', 'Density', 'Metallicity', 'hneutralssh' ]
centers = []
bins = []
dxs = []
for i, key in enumerate( histogram_axes ):
    
    arr = copy.copy( f['histogram_axes'][key][...] )
    dx = arr[2] - arr[1]
    
    # For convenience, let's not have a -inf edge lying around
    if key == 'Metallicity':
        arr[0] = arr[1] - dx
        
    centers_i = arr[:-1] + 0.5 * dx
    
    dxs.append( dx )
    centers.append( centers_i )
    bins.append( arr )

In [ ]:
# Normalize into a pdf

norm = f['histogram'][...].sum() * dxs[0] * dxs[1] * dxs[2] * dxs[3]
pdf = f['histogram'][...] / norm

## Construct Ways to Sample the Distribution

In [ ]:
# Create a Kernel Density Estimate to Allow Easy Resampling
kde = kale.KDE.from_hist(
    bins,
    pdf,
    reflect = [
        None,
        None,
        [ None, 0. ],
        None,
    ],
)

In [ ]:
# Create an interpolation function
def prob_fn( temp, den, met, hi ):
    
    return scipy.interpolate.interpn( centers, pdf, [ temp, den, met, hi ] )[0]

In [ ]:
# Find the peak
max_ind = np.argmax( pdf, )
max_inds = np.unravel_index( max_ind, f['histogram'][...].shape )
max_value = np.max( pdf )
p0 = tuple( [ centers[i][max_inds[i]] for i in range( len( max_inds ) ) ] )

In [ ]:
# Check work
print( 'If these two values are consistent then interpolation is working correctly.' )
print( '    max value = {:.3g}, max interpolated value = {:.3g}'.format( max_value, prob_fn( *p0 ) ) )

# Construct Sightline Parameters

## Construct

In [ ]:
param_labels = [ 'Temperature', 'Density', 'Metallicity', 'HI Column' ]

In [ ]:
if load_existing_sightlines:
    sls = verdict.Dict.from_hdf5( './data/synthetic_data/sample1/sightlines.h5', jagged_flag='sl' )

In [ ]:
if not load_existing_sightlines:
    sls = {
        'Temperature': [],
        'Density': [],
        'Metallicity': [],
        'HI Column': [],
        'LOS Velocity': [],
        'PDF Value': [],
    }
    for i in tqdm.tqdm( range( n_sightlines ) ):

        # Choose number of clouds
        n_clouds = rng.integers( min_clouds_per_sightline, max_clouds_per_sightline+1 )

        n_hot = 0
        n_clouds_in_sample = 0
        sample = []
        while n_clouds_in_sample < n_clouds:
            # Sample from distribution
            cloud_i = kde.resample( 1 ).transpose()[0]

            # Only allow up to one hot cloud per sightline
            if cloud_i[0] > finite_cloud_max_logT:
                if n_hot > 0:
                    continue
                n_hot += 1

            sample.append( cloud_i )
            n_clouds_in_sample += 1
        sample = np.array( sample ).transpose()

        # Store parameters
        for i, param in enumerate( param_labels ):
            sls[param].append( sample[i] )

        # Choose velocities
        sls['LOS Velocity'].append( rng.uniform( *velocity_range, size=n_clouds ) )

        # Identify frequency in simulations
        sls['PDF Value'].append( np.array([ prob_fn( *_ ) for _ in sample.transpose() ]) )

In [ ]:
clouds = {}
for key, item in sls.items():
    clouds[key] = np.concatenate( item )

## Visualize

In [ ]:
# Set up combinations to iterate through
inds = range( len( histogram_axes ) )
combinations = [ _ for _ in itertools.combinations( inds, 2 ) ]

In [ ]:
labels = [ 'Temperature (K)', r'Density (g/cm$^{3}$)', 'Metallicity (mass fraction)', r'$N_{\rm HI}$ (cm$^{-2}$)' ]
for k, (i, j) in enumerate( combinations ):
    
    # Show data
    fig = plt.figure( figsize=(8,8), facecolor='w' )
    ax = plt.gca()

    sum_axes = copy.copy( list( inds ) )
    sum_axes.remove( i )
    sum_axes.remove( j )
    projection = pdf.sum( axis=tuple( sum_axes ) ).transpose()
    x, y = np.meshgrid( centers[i], centers[j] )
    ax.pcolormesh(
        centers[i],
        centers[j],
        np.log10( projection ),
        cmap = 'cubehelix_r',
    )
    
    ax.scatter(
        clouds[param_labels[i]],
        clouds[param_labels[j]],
        edgecolor = 'k',
        s = 100,
        c = clouds['PDF Value'],
        vmin = np.nanmin( pdf[np.nonzero(pdf)] ),
        vmax = np.nanmax( pdf[np.nonzero(pdf)] ),
        cmap = palettable.cubehelix.classic_16_r.mpl_colormap,
        norm = matplotlib.colors.LogNorm(),
    )

    
    ax.set_xlabel( labels[i], fontsize=22 )
    ax.set_ylabel( labels[j], fontsize=22 )
    
    ax.set_aspect( 'auto' )
    
    plt.savefig(
        './figures/sample1/clouddist_{}_{}.png'.format(
            param_labels[i].replace( ' ', '' ),
            param_labels[j].replace( ' ', '' )
        ),
        bbox_inches = 'tight',
    )

In [ ]:
# 1D histograms
for i, param_label in enumerate( param_labels ):
    
    fig = plt.figure( figsize=(10,4), facecolor='w' )
    ax = plt.gca()
    
    sum_axes = copy.copy( list( inds ) )
    sum_axes.remove( i )
    projection = pdf.sum( axis=tuple( sum_axes ) )
    projection /= projection.sum() * dxs[i]
    
    ax.step(
        centers[i],
        projection,
        where = 'mid',
        color = 'k',
        linewidth = 3,
    )
    
    # For convenience, let's not have a -inf edge lying around
    used_bins = copy.copy( bins[i] )
    if param_label == 'Metallicity':
        used_bins[-1] = 1.0001
    
    hist, used_bins = np.histogram(
        clouds[param_labels[i]],
        bins = used_bins,
    )
    hist =  hist / ( hist.sum() * dxs[i] )
    ax.step(
        used_bins[1:],
        hist,
        where = 'pre',
        color = palettable.cartocolors.qualitative.Safe_10.mpl_colors[1],
        linewidth = 3,
    )
    
    ax.set_xlabel( labels[i], fontsize=20 )

# Create Spectra

Borrowing heavily from `one_zone.py`

In [ ]:
# Objects for use
ldb = trident.LineDatabase('lines.txt')

In [ ]:
# Add Mg II lines
ldb.add_line( 'Mg', 'II', 2796, use_linetools=True)
ldb.add_line( 'Mg', 'II', 2803, use_linetools=True)

In [ ]:
# Make sure we have the lower part of this Si IV doublet
ldb.add_line( 'Si', 'IV', 1394, use_linetools=True)

In [ ]:
sg_cos = trident.SpectrumGenerator('COS-G130M', line_database=ldb )
sg_cos_160 = trident.SpectrumGenerator('COS-G160M', line_database=ldb )

In [ ]:
# Spectrum Generator for Mg II from ground
lambda_mg = ldb.select_lines( 'Mg', 'II', 2796 )[0].wavelength * ( 1. + redshift )
sg_mg = trident.SpectrumGenerator(
    lambda_min = lambda_mg - 30.,
    lambda_max = lambda_mg + 30.,
    dlambda = 0.01,
    lsf_kernel = os.path.join( trident.path, 'data', 'lsf_kernels', 'avg_COS.txt' ),
    line_database = ldb,
)

In [ ]:
def plot_ion( sg, element, ion_state, width=6. ):
    '''Save a plot of a particular part of the spectrum for inspection.'''
    
    lines = ldb.select_lines( element, ion_state )
    
    wavelengths = np.array([ _.wavelength for _ in lines ])
    adjusted_wavelengths = wavelengths * ( 1 + redshift )
    
    data_subdir = '{}/ion_spectra/{}{}'.format( data_dir, element, ion_state, )
    os.makedirs( data_subdir, exist_ok=True )
    for k, lambda_a in enumerate( adjusted_wavelengths ):
        if lambda_a - width/2. < sg.lambda_min or lambda_a + width/2 > sg.lambda_max:
            continue
            
        sg.plot_spectrum(
            '{}/spectrum_{:.1f}_sl{:04d}.png'.format( data_subdir, lambda_a, i ),
            lambda_limits = [ lambda_a - width/2, lambda_a + width/2 ]
        )

In [ ]:
# Waaaay too much output otherwise
yt.utilities.logger.disable_stream_logging()

In [ ]:
# Loop over and create all spectra
sls['Lengths'] = []
spectrum_sg_tags = [ '_G130', '_G160', '_MgII']
for i in tqdm.tqdm( range( n_sightlines ) ):
    
    for m, sg in enumerate( [ sg_cos, sg_cos_160, sg_mg ]):

        density = 10.**sls['Density'][i] * u.g * u.cm**-3
        temperature = 10.**sls['Temperature'][i] * u.K
        metallicity = 10.**sls['Metallicity'][i] * u.Zsun / 0.014
        HI_column = 10.**sls['HI Column'][i] * u.cm**-2
        velocity = sls['LOS Velocity'][i] * u.km / u.s

        os.makedirs( os.path.join( data_dir, 'clouds' ), exist_ok=True )
        spectrum_fps = [
            '{}/clouds/spectrum_sl{:04d}_cloud{:02d}.h5'.format( data_dir, i, j )
            for j in range( density.size )
        ]

        lengths_i = []
        taus_i = []
        for j, density_j in enumerate( density ):

            # First, let's create a one-zone dataset for our desired density,
            # temperature, metallicity, and redshift.  We'll arbitrarily set it to
            # be 1 kpc in width.  
            ds = trident.make_onezone_dataset(
                density = density[j],
                temperature = temperature[j],
                metallicity = metallicity[j],
                domain_width = 1.*u.kpc
            )
            ds.current_redshift = redshift

            # Now let's add our desired ions to this dataset, using Trident's 
            # lookup table based on the Haardt-Madau 2012 UV background.
            trident.add_ion_fields(ds, ions=ions, line_database=ldb)

            # Since we now know the HI number density for this dataset, and we
            # have a desired HI column density from the simulation distribution, we can divide 
            # these two to get a desired length for the dataset.
            length = HI_column[j] / ds.r[('gas', 'H_p0_number_density')][0]
            lengths_i.append( length )

            if verbose:
                print("DEBUG: For a log HI column of %s, we require a Pathlength of %s" % 
                        (np.log10(HI_column[i]), length.to('kpc')))

            if verbose:
                # Print out the dataset to STDOUT
                one_zone.print_dataset(ray, ions, fields)

            # Correct the redshift
            z_vel = np.sqrt( ( 1 + velocity[j] / u.c) / ( 1 - velocity[j] / u.c) ) - 1.
            z_cloud = ( 1. + redshift )*( 1. + z_vel ) - 1.

            # Now that we have a length for our dataset, which will produce our 
            # desired HI column density, let's generate a one-zone
            # LightRay (skewer) using this length, density, temperature, & redshift.
            # And add the relevant ions to this dataset.
            ray = trident.make_onezone_ray(
                density = density[j],
                temperature = temperature[j],
                metallicity = metallicity[j],
                length = length,
                redshift = z_cloud,
            )
            trident.add_ion_fields(ray, ions=ions, line_database=ldb)

            sg.make_spectrum( ray, lines=ions, store_observables=True )
            sg.save_spectrum( spectrum_fps[j] )
            
            observables = sg.line_observables_dict

            d = verdict.Dict.from_hdf5( spectrum_fps[j] )
            wavelength = d['wavelength']
            taus_i.append( d['tau'] )

        lengths_i = np.array( lengths_i )
        if m == 0:
            sls['Lengths'].append( lengths_i )
        taus_i = np.array( taus_i )

        # Combine into a new spectrum
        tau_i = np.sum( taus_i, axis=0 )
        flux_i = np.exp( -tau_i )
        sg.load_spectrum( wavelength * u.Angstrom, tau_i, flux_i )

        # Noise
        sg.apply_lsf()
        sg.add_gaussian_noise( snr )

        # Save
        sg.save_spectrum(
            '{}/spectrum{}_sl{:04d}.h5'.format( observer_data_dir, spectrum_sg_tags[m], i )
        )
        sg.plot_spectrum(
            '{}/spectrum{}_sl{:04d}.png'.format( observer_data_dir, spectrum_sg_tags[m], i )
        )

        # Plot individual ions for inspection
        if m == 0:
            for ion in ions:
                plot_ion( sg, *ion.split() )

## Save Sightline Input

Now that we also have the lengths of absorbers

In [ ]:
if not load_existing_sightlines:
    sls = verdict.Dict( sls )
    sls.to_hdf5( './data/synthetic_data/sample1/sightlines.h5', handle_jagged_arrs='row datasets', jagged_flag='sl' )